# Notebook 0: Classes de Base pour la Résolution de Sudoku

Ce notebook contient les classes de base nécessaires pour la manipulation et la résolution des grilles de Sudoku. Il sera importé dans les autres notebooks pour réutiliser ces classes.

## Importation des Bibliothèques Nécessaires

Nous commençons par importer les bibliothèques nécessaires.


In [ ]:
using System;
using System.Collections.Generic;
using System.Collections.ObjectModel;
using System.IO;
using System.Linq;
using System.Text;


## Définition de la classe SudokuGrid

Nous définissons ici la classe SudokuGrid qui représente une grille de Sudoku et fournit des méthodes pour manipuler et afficher les grilles.


In [ ]:
public class SudokuGrid : ICloneable
{

    // Méthodes utilitaires
    public static T[][] ToJaggedArray<T>(IList<T> source, int columnLength)
    {
        var rowLength = source.Count / columnLength;
        var toReturn = new T[rowLength][];
        for (int rowIndex = 0; rowIndex < rowLength; rowIndex++)
        {
            toReturn[rowIndex] = new T[columnLength];
            for (int colIndex = 0; colIndex < columnLength; colIndex++)
            {
                var globalIndex = rowIndex * columnLength + colIndex;
                if (globalIndex < source.Count)
                {
                    toReturn[rowIndex][colIndex] = source[globalIndex];
                }
            }
        }

        return toReturn;
    }

    public static T[] Flatten<T>(T[][] source)
    {
        return source.SelectMany(x => x).ToArray();
    }

    public static T[,] To2D<T>(T[][] source)
    {
        try
        {
            int FirstDim = source.Length;
            int SecondDim = source.GroupBy(row => row.Length).Single().Key;

            var result = new T[FirstDim, SecondDim];
            for (int i = 0; i < FirstDim; ++i)
                for (int j = 0; j < SecondDim; ++j)
                    result[i, j] = source[i][j];

            return result;
        }
        catch (InvalidOperationException)
        {
            throw new InvalidOperationException("The given jagged array is not rectangular.");
        }
    }

    public static T[][] ToJaggedArray<T>(T[,] twoDimensionalArray)
    {
        int rowsFirstIndex = twoDimensionalArray.GetLowerBound(0);
        int rowsLastIndex = twoDimensionalArray.GetUpperBound(0);
        int numberOfRows = rowsLastIndex + 1;

        int columnsFirstIndex = twoDimensionalArray.GetLowerBound(1);
        int columnsLastIndex = twoDimensionalArray.GetUpperBound(1);
        int numberOfColumns = columnsLastIndex + 1;

        T[][] jaggedArray = new T[numberOfRows][];
        for (int i = rowsFirstIndex; i <= rowsLastIndex; i++)
        {
            jaggedArray[i] = new T[numberOfColumns];
            for (int j = columnsFirstIndex; j <= columnsLastIndex; j++)
            {
                jaggedArray[i][j] = twoDimensionalArray[i, j];
            }
        }
        return jaggedArray;
    }


    public static readonly ReadOnlyCollection<int> NeighbourIndices =
        new ReadOnlyCollection<int>(Enumerable.Range(0, 9).ToList());

    private static readonly (int row, int column)[][] _LineNeighbours =
        NeighbourIndices.Select(r => NeighbourIndices.Select(c => (r, c)).ToArray()).ToArray();

    private static readonly (int row, int column)[][] _ColumnNeighbours =
        NeighbourIndices.Select(c => NeighbourIndices.Select(r => (r, c)).ToArray()).ToArray();

    private static readonly (int row, int column)[][] _BoxNeighbours = GetBoxNeighbours();

    public static readonly (int row, int column)[][] AllNeighbours =
        _LineNeighbours.Concat(_ColumnNeighbours).Concat(_BoxNeighbours).ToArray();

    private static (int row, int column)[][] GetBoxNeighbours()
    {
        var toreturn = new (int row, int column)[9][];
        for (int boxIndex = 0; boxIndex < 9; boxIndex++)
        {
            var currentBox = new List<(int row, int column)>();
            (int row, int column) startIndex = (boxIndex / 3 * 3, boxIndex % 3 * 3);
            for (int r = 0; r < 3; r++)
            {
                for (int c = 0; c < 3; c++)
                {
                    currentBox.Add((startIndex.row + r, startIndex.column + c));
                }
            }
            toreturn[boxIndex] = currentBox.ToArray();
        }
        return toreturn;
    }

    public static readonly (int row, int column)[][][] CellNeighbours;

    static SudokuGrid()
    {
        CellNeighbours = new (int row, int column)[9][][];
        foreach (var rowIndex in NeighbourIndices)
        {
            CellNeighbours[rowIndex] = new (int row, int column)[9][];
            foreach (var columnIndex in NeighbourIndices)
            {
                var cellVoisinage = new List<(int row, int column)>();
                foreach (var voisinage in AllNeighbours)
                {
                    if (voisinage.Contains((rowIndex, columnIndex)))
                    {
                        foreach (var voisin in voisinage)
                        {
                            if (!cellVoisinage.Contains(voisin) && voisin.row != rowIndex || voisin.column != columnIndex)
                            {
                                cellVoisinage.Add(voisin);
                            }
                        }
                    }
                }
                CellNeighbours[rowIndex][columnIndex] = cellVoisinage.ToArray();
            }
        }
    }

    public SudokuGrid() { }

    public int[,] Cells { get; set; } = To2D(NeighbourIndices.Select(r => new int[9]).ToArray());

    public override string ToString()
    {
        var lineSep = new string('-', 31);
        var blankSep = new string(' ', 8);

        var output = new StringBuilder();
        output.Append(lineSep);
        output.AppendLine();

        for (int row = 1; row <= 9; row++)
        {
            output.Append("| ");
            for (int column = 1; column <= 9; column++)
            {
                var value = Cells[row - 1, column - 1];
                output.Append(value);
                if (column % 3 == 0)
                {
                    output.Append(" | ");
                }
                else
                {
                    output.Append("  ");
                }
            }
            output.AppendLine();
            if (row % 3 == 0)
            {
                output.Append(lineSep);
            }
            else
            {
                output.Append("| ");
                for (int i = 0; i < 3; i++)
                {
                    output.Append(blankSep);
                    output.Append("| ");
                }
            }
            output.AppendLine();
        }
        return output.ToString();
    }

    public int[] GetAvailableNumbers(int x, int y)
    {
        if (x < 0 || x >= 9 || y < 0 || y >= 9)
        {
            throw new ApplicationException("Invalid Coordinates");
        }

        bool[] used = new bool[9];
        foreach (var cellNeighbour in CellNeighbours[x][y])
        {
            var neighbourValue = Cells[cellNeighbour.row, cellNeighbour.column];
            if (neighbourValue > 0)
            {
                used[neighbourValue - 1] = true;
            }
        }

        List<int> res = new List<int>();

        for (int i = 0; i < 9; i++)
        {
            if (used[i] == false)
            {
                res.Add(i + 1);
            }
        }

        return res.ToArray();
    }

    public static SudokuGrid ReadSudoku(string sudokuAsString)
    {
        return ReadMultiSudoku(new[] { sudokuAsString })[0];
    }

    public static List<SudokuGrid> ReadSudokuFile(string fileName)
    {
        return ReadMultiSudoku(File.ReadAllLines(fileName));
    }

    public static List<SudokuGrid> ReadMultiSudoku(string[] lines)
    {
        var toReturn = new List<SudokuGrid>();
        var rows = new List<int[]>();
        var rowCells = new List<int>(9);

        foreach (var line in lines.Where(l => l.Length > 0 && IsSudokuChar(l[0])))
        {
            foreach (char c in line)
            {
                if (IsSudokuChar(c))
                {
                    if (char.IsDigit(c))
                    {
                        rowCells.Add((int)Char.GetNumericValue(c));
                    }
                    else
                    {
                        rowCells.Add(0);
                    }
                }

                if (rowCells.Count == 9)
                {
                    rows.Add(rowCells.ToArray());
                    rowCells.Clear();
                }

                if (rows.Count == 9)
                {
                    toReturn.Add(new SudokuGrid() { Cells = To2D(rows.ToArray()) });
                    rows.Clear();
                }
            }
        }

        return toReturn;
    }

    private static bool IsSudokuChar(char c)
    {
        return char.IsDigit(c) || c == '.' || c == 'X' || c == '-';
    }

    public object Clone()
    {
        return CloneSudoku();
    }

    public SudokuGrid CloneSudoku()
    {
        var toReturn = new SudokuGrid();
        toReturn.Cells = (int[,])Cells.Clone();
        return toReturn;
    }

    public int NbErrors(SudokuGrid originalPuzzle)
    {
        var toReturn = SudokuGrid.AllNeighbours.Select(n => n.Select(nx => this.Cells[nx.row, nx.column]))
            .Sum(n => n.GroupBy(x => x).Select(g => g.Count() - 1).Sum());

        foreach (var rowIndex in NeighbourIndices)
        {
            foreach (var colIndex in NeighbourIndices)
            {
                if (originalPuzzle.Cells[rowIndex, colIndex] > 0 && originalPuzzle.Cells[rowIndex, colIndex] != Cells[rowIndex, colIndex])
                {
                    toReturn += 1;
                }
            }
        }

        return toReturn;
    }

    public bool IsValid(SudokuGrid originalPuzzle)
    {
        return NbErrors(originalPuzzle) == 0;
    }

    public int NbEmptyCells() => Cells.Cast<int>().Count(c => c == 0);
}


## Définition de l'interface ISudokuSolver

Nous définissons ici l'interface ISudokuSolver qui sera implémentée par les différentes stratégies de résolution de Sudoku.


In [ ]:
public interface ISudokuSolver
{
    SudokuGrid Solve(SudokuGrid s);
}


## Définition de la classe SudokuHelper

Nous ajoutons ici la classe SudokuHelper qui contient des méthodes utilitaires pour charger et afficher des grilles de Sudoku.


In [ ]:
using System.IO;

public enum SudokuDifficulty
{
    Easy,
    Medium,
    Hard
}

public static class SudokuHelper
{
    private const string PUZZLES_FOLDER_NAME = "Puzzles";

    public static List<SudokuGrid>  GetSudokus(SudokuDifficulty difficulty)
    {
        string fileName = difficulty switch
        {
            SudokuDifficulty.Easy => "Sudoku_Easy51.txt",
            SudokuDifficulty.Medium => "Sudoku_hardest.txt",
            _ => "Sudoku_top95.txt"
        };

        DirectoryInfo puzzlesDirectory = null;
        var currentDirectory = new DirectoryInfo(Environment.CurrentDirectory);
        do
        {
            var subDirectories = currentDirectory.GetDirectories();
            foreach (var subDirectory in subDirectories)
            {
                if (subDirectory.Name == PUZZLES_FOLDER_NAME)
                {
                    puzzlesDirectory = subDirectory;
                    break;
                }
            }
            currentDirectory = currentDirectory.Parent;
            if (currentDirectory == null)
            {
                throw new ApplicationException("couldn't find puzzles directory");
            }
        } while (puzzlesDirectory == null);
        string filePath = System.IO.Path.Combine(puzzlesDirectory.ToString(), fileName);
        var sudokus = SudokuGrid.ReadSudokuFile(filePath);
        return sudokus;
    }


    public static SudokuGrid SolveSudoku(SudokuGrid sudoku, ISudokuSolver solver)
    {
        
        display($"Résolution par le solver {solver.GetType().Name} du Sudoku:\n {sudoku.ToString()}");
        var stopWatch = System.Diagnostics.Stopwatch.StartNew();
        var solvedSudoku = solver.Solve(sudoku);
        stopWatch.Stop();
        var durationEasySolver = stopWatch.Elapsed.TotalMilliseconds;
        display($"Sudoku renvoyé: \n{solvedSudoku}\nNombre d'erreurs réstantes: {solvedSudoku.NbErrors(sudoku)}\nTemps de résolution : {durationEasySolver} ms");
        return solvedSudoku;
    }
}


## Affichage des Puzzles de chaque Difficulté

Nous allons charger et afficher un puzzle de chaque niveau de difficulté : Facile, Moyen et Difficile.


In [ ]:

// Chargement et affichage d'un puzzle facile
var easySudokus = SudokuHelper.GetSudokus(SudokuDifficulty.Easy);
var easySudoku = easySudokus.FirstOrDefault();
display($"Puzzle Facile:\n{easySudoku.ToString()}");

// Chargement et affichage d'un puzzle moyen
var mediumSudokus = SudokuHelper.GetSudokus(SudokuDifficulty.Medium);
var mediumSudoku = mediumSudokus.FirstOrDefault();
display($"Puzzle Moyen:\n{mediumSudoku.ToString()}");

// Chargement et affichage d'un puzzle difficile
var hardSudokus = SudokuHelper.GetSudokus(SudokuDifficulty.Hard);
var hardSudoku = hardSudokus.FirstOrDefault();
display($"Puzzle Difficile:\n{hardSudoku.ToString()}");
